"Electric Reliability Council of Texas (ERCOT) manages the flow of electric power to more than 25 million Texas customers -- representing about 90 percent of the state’s electric load." (<a href="http://www.ercot.com/about">source</a>)

Example report: http://www.ercot.com/content/wcm/lists/143010/2018_Long-Term_Hourly_Peak_Demand_and_Energy_Forecast_Final.pdf

Data source:<BR>
http://www.ercot.com/gridinfo/load/load_hist/

In [ ]:
import pandas as pd
print('pandas',pd.__version__)
import numpy
print('numpy',numpy.__version__)
import glob
from scipy.fftpack import fft, ifft
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
import random
import time
import matplotlib.pyplot as plt

In [ ]:
start_time=time.time()
df = pd.read_pickle('../data/power_data.pkl')
print(df.shape)
print('elapsed:',time.time()-start_time,'seconds')

In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
df.isna().sum() # my fault is that I didn't inspect this and spent literally hours to make sense of why my plot doesn't make sense

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.dtypes

In [ ]:
df['Hour_End'].dt.year.unique()

# visualize data

scatter plot doesn't work with dates

df.plot.scatter(x='Hour_End',y='EAST')

use plot_date to see all the data

In [ ]:
plt.plot_date(x=df['Hour_End'],y=df['EAST'],markersize=1);

To get a sense of the data contents, zoom in on the first few data points

The sampling is once per hour, so to get 5 days we an look at the first 24*5 data points

In [ ]:
max_ct = 24*5
plt.plot_date(x=df['Hour_End'][0:max_ct],y=df['EAST'][0:max_ct],markersize=5);
plt.xticks(rotation=60);

Take a look at the first month. We can see there are weekly patterns

In [ ]:
max_ct = 24*30
plt.plot_date(x=df['Hour_End'][0:max_ct],y=df['EAST'][0:max_ct],markersize=5);
plt.xticks(rotation=60);

As expected, there are daily, weekly, and annual patterns

In [ ]:
max_ct = 24*365
plt.plot_date(x=df['Hour_End'][0:max_ct],y=df['EAST'][0:max_ct],markersize=5);
plt.xticks(rotation=60);

There are 9 columns, one per region of monitoring
 

We can plot each of these columns to see the differences

In [ ]:
for column_name in df.columns:
    if column_name not in ['Hour_End', 'index']:
        plt.plot_date(x=df['Hour_End'],y=df[column_name],label=column_name,markersize=2)
        plt.title(column_name)
        plt.show()

# lag plot - compare each point to previous point

If data isn't a time series, we would see a circular blog of points, indicating order doesn't matter

In [ ]:
pd.plotting.lag_plot(df['EAST']);

# autocorrelation measures many lags

See the topic https://en.wikipedia.org/wiki/Autocorrelation, specifically here we are creating a https://en.wikipedia.org/wiki/Correlogram

"Calculate the correlation for time series observations with observations with previous time steps, called lags. Because the correlation of the time series observations is calculated with values of the same series at previous times, this is called a serial correlation, or an autocorrelation." (<a href="https://machinelearningmastery.com/gentle-introduction-autocorrelation-partial-autocorrelation/">source</a>)

"If time series is random, such autocorrelations should be near zero for any and all time-lag separations." (<a href="https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html#visualization-autocorrelation">source</a>)

https://data.blog/2018/07/24/investigating-seasonality-in-a-time-series-a-mystery-in-three-parts/

In [ ]:
start_time=time.time()
pd.plotting.autocorrelation_plot(df['EAST']);
print('elapsed:',time.time()-start_time,'seconds')

A similar option is to use the statsmodels library

https://www.statsmodels.org/dev/generated/statsmodels.graphics.tsaplots.plot_acf.html

"Confidence intervals are drawn as a cone. By default, this is set to a 95% confidence interval, suggesting that correlation values outside of this code are very likely a correlation and not a statistical fluke."" (<a href="https://machinelearningmastery.com/gentle-introduction-autocorrelation-partial-autocorrelation/">source</a>)

In [ ]:
_=plot_acf(df['EAST'])

we can zoom in on the correlations that are outside the cone

In [ ]:
_=plot_acf(df['EAST'],lags=40000)

In [ ]:
365*24

# temporal decomposition

In [ ]:
df_2020 = df[df['Hour_End'].apply(lambda x: x.year==2020)]

In [ ]:
df_2020.head()

# smoothing out noise using rolling average

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html

In [ ]:
df['EAST'].rolling(window=5).mean().head(15)

In [ ]:
# window size of one week (7 days and 24 hours)
plt.plot_date(x=df['Hour_End'],y=df['EAST'].rolling(window=7*24).mean(),markersize=2);

In [ ]:
# window size of one month (30 days and 24 hours)
plt.plot_date(x=df['Hour_End'],y=df['EAST'].rolling(window=30*24).mean(),markersize=2);

In [ ]:
# window size of two months (60 days and 24 hours)
plt.plot_date(x=df['Hour_End'],y=df['EAST'].rolling(window=60*24).mean(),markersize=2);

In [ ]:
plt.plot_date(x=df['Hour_End'],y=df['EAST'].rolling(window=365*12).mean(),markersize=2);

### A window that is too large loses the signal and the noise

In [ ]:
# window size of one year (365 days and 24 hours)
plt.plot_date(x=df['Hour_End'],y=df['EAST'].rolling(window=365*24).mean(),markersize=2);

# Seasonal decomposition

https://www.statsmodels.org/dev/generated/statsmodels.tsa.seasonal.seasonal_decompose.html

https://github.com/statsmodels/statsmodels/blob/master/statsmodels/tsa/seasonal.py

First, a reminder of the contents of Hour_End column

In [ ]:
df['Hour_End'][0]

the sampling is once per hour

In [ ]:
df['Hour_End'][0:5]

Let's inspect the "EAST" columns versus Hour_End.

To get a picture with timestamps on the x-axis, we need to convert the twol columns to a series with a timestamp as the index

In [ ]:
data_series = pd.Series(df['EAST'].values, index=df['Hour_End'])
data_series.head()

Now we can pass the series to seasonal_decompose and inspect the first 10 days

In [ ]:
result  = seasonal_decompose(data_series[0:24*10], model='additive',freq=24) # freq is the cycle length in number of periods
result.plot()
plt.gcf().set_size_inches(10,8)
plt.show()

We can include more data points to see a weekly trend

In [ ]:
result  = seasonal_decompose(data_series[0:24*30], model='additive',freq=24) # freq is the cycle length in number of periods
result.plot()
plt.gcf().set_size_inches(10,8)
plt.show()

In [ ]:
result  = seasonal_decompose(data_series[0:24*30], model='additive',freq=24*7) # freq is the cycle length in number of periods
result.plot()
plt.gcf().set_size_inches(10,8)
plt.show()

Next we can filter out only values where the year is 2020

In [ ]:
data_series_2020 = pd.Series(df_2020['EAST'].values, index=df_2020['Hour_End'])
data_series_2020.head()

In [ ]:
result  = seasonal_decompose(data_series_2020, model='additive',freq=365) # freq is the cycle length in number of periods
result.plot()
plt.gcf().set_size_inches(15,8)
plt.show()